In [13]:
"""
Defines and runs all the experiments in our paper:
Multilevel domain decomposition-based architectures for physics-informed neural networks
https://arxiv.org/abs/2306.05486
"""

import numpy as np

from fbpinns.constants import Constants, get_subdomain_ws
from fbpinns.domains import RectangularDomainND
from fbpinns.decompositions import MultilevelRectangularDecompositionND, RectangularDecompositionND
from fbpinns.networks import FCN, SIREN, FourierFCN
from fbpinns.util.sbatch import submit

# from problems import Laplace1D_quadratic, Laplace2D_quadratic, Laplace2D_multiscale, Helmholtz2D


def pscan(p0, *pss):
    "scan from fixed point"
    assert len(p0) == len(pss)
    return [list(p0[:ip] + [p] + p0[ip+1:]) for ip, ps in enumerate(pss) for p in ps]


runs=[]
model_save_freq=10000
n_seeds=10


## TEST 1: simple ablation tests

tag = "ablation"

test_freq=500

# Laplace1D_quadratic ablation
domain=RectangularDomainND
domain_init_kwargs=dict(xmin=np.array([0.,]),
                        xmax=np.array([1.,]),)
# problem=Laplace1D_quadratic
problem_init_kwargs=dict()
unnorm=(0.5, 0.375)# unnorm

n_steps=20000# number training steps
n=(80,)# number training points
n_test=(350,)# number test points
lr = 1e-3# learning rate

h=1

ls=[2, 3, 4, 5]# number of levels
ws=[1.1, 1.5, 1.9, 2.3, 2.7]# overlap width
ps=[2, 4, 8, 16, 32]# number of hidden units
p0=[3, 1.9, 16]

tag = "strong"

# problem=Laplace2D_multiscale
omegas=[2, 4, 8, 16, 32, 64]
unnorm=(0., 0.75)# unnorm

n_steps=30000# number training steps
n = (320,320)# number training points
n_test=(350,350)# number test points
lr = 1e-3# learning rate

h=1
w=2
p=16

# increase levels, but not collocation points
ls=[3, 4, 5, 6, 7]# number of levels
for il,l_ in enumerate(ls):
    problem_init_kwargs=dict(omegas=omegas, sd=1/omegas[-1])

    # multilevel scaling
    l = [2**i for i in range(l_)]
    subdomain_xss = [[np.array([1.5]),np.array([1.5])]] + [[np.linspace(0,3,n_),np.linspace(0,3,n_)] for n_ in (4,)]
    subdomain_wss = [[np.array([w*1.]),np.array([w*1.])]] + [get_subdomain_ws(subdomain_xs, w) for subdomain_xs in subdomain_xss[1:]]
    
    print(subdomain_xss)
    print(subdomain_wss)
    break


[[array([1.5]), array([1.5])], [array([0., 1., 2., 3.]), array([0., 1., 2., 3.])]]
[[array([2.]), array([2.])], [array([2., 2., 2., 2.]), array([2., 2., 2., 2.])]]
